In [13]:
from IPython.display import display
from IPython.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import hail as hl
import sys
import os

hl.init(tmp_dir=os.path.join(os.environ.get('SCRATCH'), 'hail-tmpdir'),
 local_tmpdir=os.path.join(os.environ.get('SCRATCH_LOCAL'), 'hail-local-tmpdir'),
 spark_conf={'spark.driver.memory': '20G', 'spark.executor.memory': '20G'}, default_reference='GRCh38') 

2022-03-10 14:53:32 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.2
SparkUI available at http://p1708.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.85-9b98676b6ad8
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/analysis/hail-20220310-1453-0.2.85-9b98676b6ad8.log


In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats
import seaborn as sns

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [3]:
mt = hl.import_vcf('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/mitocarta.vcf.bgz')

In [4]:
mt.write('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/mitocarta.mt')

2022-03-09 16:43:47 Hail: INFO: Coerced sorted dataset              (1 + 1) / 2]
2022-03-09 16:44:12 Hail: INFO: wrote matrix table with 88127 rows and 288 columns in 2 partitions to /net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/mitocarta.mt


In [38]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/mitocarta.mt')

In [39]:
controls = [ 'S_7212','S_7213','S_7227','S_7255','S_7237','S_7245','S_7246','S_7229','S_7254','WGS_147c','S_7261','S_7263','S_7269','S_7274','S_7294','S_7306','WGS_37b','WGS_37c','WGS_85b','WGS_7118',
'WGS_7120','WGS_7142','WGS_7143','WGS_7152','WGS_7153','WGS_163d','WGS_180b','WGS_6819','WGS_D6813','WGS_D6815','462','468','475','476','477','478','479','482','490','492','494']

In [40]:
sport_pheno = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/data/external-data/sportsmen-pheno.csv', impute = True, key='sample_id', quote ="\"")
mt = mt.annotate_cols(sport_phenotypes = sport_pheno[mt.s])

2022-03-10 18:23:52 Hail: INFO: Reading table to impute column types
2022-03-10 18:23:52 Hail: INFO: Finished type imputation
  Loading field 'sport' as type str (imputed)
  Loading field 'type' as type str (imputed)
  Loading field 'age' as type int32 (imputed)
  Loading field 'sample_id' as type str (imputed)


In [41]:
mt = mt.annotate_cols(group = hl.if_else(mt.s.contains('B'), hl.if_else(mt.sport_phenotypes.type == 'speed', 'speed', 'endurance'), hl.if_else(
                                         hl.literal(controls).contains(mt.s), 'controls', 'GTS')))

In [43]:
#run fisher for endurance & speed vs controls 

control_AC_non_ref = mt.info.het_controls + (mt.info.hom_var_controls*2)
control_AC_ref = mt.info.het_controls + (mt.info.hom_ref_controls*2)

speed_AC_non_ref = mt.info.het_speed + (mt.info.hom_var_speed*2)
speed_AC_ref = mt.info.het_speed + (mt.info.hom_ref_speed*2)
endurance_AC_non_ref = mt.info.het_endurance + (mt.info.hom_var_endurance*2)
endurance_AC_ref = mt.info.het_endurance + (mt.info.hom_ref_endurance*2)

mt = mt.annotate_rows(fisher_e_vs_control = hl.fisher_exact_test(endurance_AC_non_ref, endurance_AC_ref, control_AC_non_ref, control_AC_ref),
                      fisher_s_vs_control = hl.fisher_exact_test(speed_AC_non_ref, speed_AC_ref, control_AC_non_ref, control_AC_ref))

In [44]:
mt = mt.annotate_rows(info = mt.info.annotate(fisher_endurance_vs_control = mt.fisher_e_vs_control.p_value,
                                              fisher_speed_vs_control = mt.fisher_s_vs_control.p_value))

In [45]:
mt = mt.drop(mt.fisher_e_vs_control, mt.fisher_s_vs_control)

In [46]:
mt = mt.rows()

In [47]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        DP_stats: str, 
        GQ_stats: str, 
        gnomad_v3_nfe_af: float64, 
        gnomad_v3_nfe_homozygote_count: int32, 
        cadd: float64, 
        within_gene: array<str>, 
        hpo: str, 
        most_severe_consequence: str, 
        transcript_consequences: str, 
        intergenic_consequences: str, 
        motif_feature_consequences: str, 
        regulatory_feature_consequences: str, 
        fisher_sport_vs_control: str, 
        fisher_e_vs_s: str, 
        fisher_sport_vs_gnomad: str, 
        fisher_sport_vs_polish: str, 
        het_sport: int32, 
        het_controls: int32, 
        hom_ref_sport: int32, 
        hom_ref_controls: int32, 
        hom_var_sport: int32, 
        hom_var_controls: int32

In [48]:
# save top 100 p_values for additional comparisons:

end_vs_contr_p = mt.order_by(mt.info.fisher_endurance_vs_control).take(100)[99] #0.0010648572464885136
speed_vs_contr_p = mt.order_by(mt.info.fisher_speed_vs_control).take(100)[99] #0.002792860705231459

In [64]:
to_export_1 = mt.filter(mt.info.fisher_endurance_vs_control <= 0.0010648572464885136).to_pandas()
to_export_2 = mt.filter(mt.info.fisher_speed_vs_control <= 0.002792860705231459).to_pandas()

In [65]:
to_export_1.shape

(119, 36)

In [66]:
to_export_2.shape

(100, 36)

In [67]:
to_export_1.to_csv('endurance_vs_controls.csv')
to_export_1.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/endurance_vs_controls.csv')

In [68]:
to_export_2.to_csv('speed_vs_controls.csv')
to_export_2.to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/speed_vs_controls.csv')

## Now export only coding variants:

In [69]:
coding_description = ['missense_variant', 'start_lost', 'stop_gained', 'stop_lost', 'stop_retained_variant', 'synonymous_variant']

In [70]:
coding = mt.filter(hl.literal(coding_description).contains(mt.info.most_severe_consequence))

In [73]:
coding.count()

3731

In [87]:
coding.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        DP_stats: str, 
        GQ_stats: str, 
        gnomad_v3_nfe_af: float64, 
        gnomad_v3_nfe_homozygote_count: int32, 
        cadd: float64, 
        within_gene: array<str>, 
        hpo: str, 
        most_severe_consequence: str, 
        transcript_consequences: str, 
        intergenic_consequences: str, 
        motif_feature_consequences: str, 
        regulatory_feature_consequences: str, 
        fisher_sport_vs_control: str, 
        fisher_e_vs_s: str, 
        fisher_sport_vs_gnomad: str, 
        fisher_sport_vs_polish: str, 
        het_sport: int32, 
        het_controls: int32, 
        hom_ref_sport: int32, 
        hom_ref_controls: int32, 
        hom_var_sport: int32, 
        hom_var_controls: int32

In [89]:
coding = coding.annotate(fisher_sport_vs_control_p_value = hl.parse_json(coding.info.fisher_sport_vs_control,
                                                           dtype='struct{p_value: float64, odds_ratio: float64, ci_95_lower: float64, ci_95_upper: float64}').p_value,
                         fisher_e_vs_s_p_value = hl.parse_json(coding.info.fisher_e_vs_s,
                                                           dtype='struct{p_value: float64, odds_ratio: float64, ci_95_lower: float64, ci_95_upper: float64}').p_value,
                         fisher_sport_vs_gnomad_p_value = hl.parse_json(coding.info.fisher_sport_vs_gnomad,
                                                           dtype='struct{p_value: float64, odds_ratio: float64, ci_95_lower: float64, ci_95_upper: float64}').p_value,
                         fisher_sport_vs_polish_p_value = hl.parse_json(coding.info.fisher_sport_vs_polish,
                                                           dtype='struct{p_value: float64, odds_ratio: float64, ci_95_lower: float64, ci_95_upper: float64}').p_value)


In [91]:
coding.to_pandas().to_csv('all-coding.csv')
coding.to_pandas().to_csv('/net/archive/groups/plggneuromol/imdik-zekanowski-sportwgs/results/mitocarta/all-coding.csv')